# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report Code
**_Opening a New Gym in Chicago, Illinois USA_**
- Build a dataframe of neighborhoods in Chicago, Illinois by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new gym
***
### 1. Import  necessary libraries

In [ ]:
!pip install geocoder

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 

from geopy.geocoders import Nominatim 
import geocoder 

import requests 
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium 

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into Pandas Dataframe

In [2]:
# send the GET request
data = requests.get("https://simple.wikipedia.org/wiki/Category:Suburbs_of_Chicago,_Illinois").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,"Addison, Illinois"
1,"Algonquin, Illinois"
2,"Arlington Heights, Illinois"
3,"Aurora, Illinois"
4,"Barrington Hills, Illinois"


In [7]:
# print the number of rows of the dataframe
kl_df.shape
kl_df

,Neighborhood
0,"Addison, Illinois"
1,"Algonquin, Illinois"
2,"Arlington Heights, Illinois"
3,"Aurora, Illinois"
4,"Barrington Hills, Illinois"
5,"Barrington, Illinois"
6,"Bartlett, Illinois"
7,"Bedford Park, Illinois"
8,"Bellwood, Illinois"
9,"Bensenville, Illinois"


## removing uncecessary text, and observation

In [8]:
kl_df['Neighborhood'] = kl_df['Neighborhood'].str.rstrip('Illinois')

In [9]:
kl_df['Neighborhood'] = kl_df['Neighborhood'].str.rstrip('(Chicago)')

In [10]:
kl_df=kl_df.drop(93)

In [11]:
#kl_df['Neighborhood'] = kl_df['Neighborhood'].str.rstrip(',')
kl_df['Neighborhood'] = kl_df['Neighborhood'].str.replace(',', '')


In [12]:
kl_df

,Neighborhood
0,Addison
1,Algonquin
2,Arlington Heights
3,Aurora
4,Barrington Hills
5,Barrington
6,Bartlett
7,Bedford Park
8,Bellwood
9,Bensenville


### 3. Get the geographical coordinates

In [13]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chicago, Illinois'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [15]:
coords

[[41.946669000000014, -87.70286334699385],
 [42.006096115810806, -87.7628525522498],
 [41.92674204846768, -87.64270688114199],
 [41.883861543067184, -88.20865435672738],
 [42.11430000000007, -88.18250999999998],
 [41.98959873536598, -87.73533006127293],
 [41.991820000000075, -88.18458999999996],
 [41.76293000000004, -87.79494999999997],
 [41.88192000000004, -87.86739999999998],
 [41.95604000000003, -87.94105999999994],
 [41.81649256537855, -87.60063784465261],
 [41.974029, -87.87109921372286],
 [41.89938002611217, -87.62414001341614],
 [41.64818967454075, -87.63280056867498],
 [41.698210000000074, -88.07963999999998],
 [41.91799000000003, -87.64853003332549],
 [41.86171000000007, -87.86273999999997],
 [41.82295000000005, -87.84058999999996],
 [41.7424813894096, -87.54421170181466],
 [41.884250000000065, -87.63244999999995],
 [41.63795897866495, -87.53958646129553],
 [41.70315000000005, -87.77931999999998],
 [41.73124000000007, -87.55125999999996],
 [41.71490000000006, -87.5316399999999

In [16]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [17]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [18]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(143, 3)


,Neighborhood,Latitude,Longitude
0,Addison,41.946669,-87.702863
1,Algonquin,42.006096,-87.762853
2,Arlington Heights,41.926742,-87.642707
3,Aurora,41.883862,-88.208654
4,Barrington Hills,42.114300,-88.182510
5,Barrington,41.989599,-87.735330
6,Bartlett,41.991820,-88.184590
7,Bedford Park,41.762930,-87.794950
8,Bellwood,41.881920,-87.867400
9,Bensenville,41.956040,-87.941060


In [19]:
kl_df=kl_df.drop(143)

In [20]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

### 4. Create a map of Chicago, Illinois with neighborhoods superimposed on top

In [21]:
# get the coordinates of the city
address = 'Chicago, Illinois'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago Illinois, USA {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago Illinois, USA 41.8755616, -87.6244212.


In [22]:
# create map of chi-town using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5).add_to(map_kl)  
    
map_kl

In [23]:
# save the map as HTML file
map_kl.save('map_kl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [50]:
CLIENT_ID = '11HZ5QGHAEFP0FF2WOUTTLBG42CCTWUUPIHSAEUODDNWH0PJ' # your Foursquare ID
CLIENT_SECRET = 'LFZW2QSBEENKE51VLQBFOL5QJCLBVB0KB1QTEEHG5D1X5SHE' # your Foursquare Secret
VERSION = '20200624' # Foursquare API version

print('sd54347n@pace.edu:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

sd54347n@pace.edu:
CLIENT_ID: 11HZ5QGHAEFP0FF2WOUTTLBG42CCTWUUPIHSAEUODDNWH0PJ
CLIENT_SECRET:LFZW2QSBEENKE51VLQBFOL5QJCLBVB0KB1QTEEHG5D1X5SHE


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [51]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [52]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(10661, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Addison,41.946669,-87.702863,Chief O'Neill's Pub & Restaurant,41.944516,-87.705544,Pub
1,Addison,41.946669,-87.702863,Parachute,41.944851,-87.706331,Korean Restaurant
2,Addison,41.946669,-87.702863,Honey Butter Fried Chicken,41.942678,-87.702854,Fried Chicken Joint
3,Addison,41.946669,-87.702863,Ignite Gaming Lounge,41.942133,-87.702259,Gaming Cafe
4,Addison,41.946669,-87.702863,Brew & Grow,41.947459,-87.707791,Hobby Shop


**Let's check how many venues were returned for each neighorhood**

In [53]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Addison,100,100,100,100,100,100
Algonquin,93,93,93,93,93,93
Arlington Heights,100,100,100,100,100,100
Aurora,25,25,25,25,25,25
Barrington,93,93,93,93,93,93
Barrington Hills,4,4,4,4,4,4
Bartlett,37,37,37,37,37,37
Bedford Park,37,37,37,37,37,37
Bellwood,49,49,49,49,49,49


**Let's find out how many unique categories can be curated from all the returned venues**

In [54]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 374 uniques categories.


In [55]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Pub', 'Korean Restaurant', 'Fried Chicken Joint', 'Gaming Cafe',
       'Hobby Shop', 'Discount Store', 'Brewery', 'Climbing Gym',
       'Athletics & Sports', 'Big Box Store', 'Tennis Court',
       'Liquor Store', 'Soccer Field', 'Burger Joint', 'Beer Store',
       'German Restaurant', 'Italian Restaurant', 'Supermarket',
       'Chinese Restaurant', 'Mexican Restaurant', 'Gymnastics Gym',
       'Restaurant', 'Skating Rink', 'Pizza Place',
       'Fast Food Restaurant', 'Football Stadium', 'Baseball Stadium',
       'Park', 'Asian Restaurant', 'Bar', 'Grocery Store', 'Dessert Shop',
       'Nail Salon', 'Dance Studio', 'Coffee Shop', 'Massage Studio',
       'Thai Restaurant', 'Food Truck', 'Theater', 'Taco Place',
       'Breakfast Spot', 'Café', 'Argentinian Restaurant',
       'South American Restaurant', 'Recreation Center', 'Bakery',
       'Tiki Bar', 'Cuban Restaurant', 'Concert Hall',
       'Gym / Fitness Center'], dtype=object)

In [56]:
# check if the results contain "Gym"
"Gym" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [57]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(10661, 375)


,Neighborhoods,ATM,Accessories Store,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bavarian Restaurant,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Car Wash,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Administrative Building,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Rink,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pawn Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pri

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [58]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(142, 375)


,Neighborhoods,ATM,Accessories Store,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bavarian Restaurant,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Car Wash,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Administrative Building,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Rink,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pawn Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pri

In [74]:
len(kl_grouped[kl_grouped["Gym"] > 0])

78

**Create a new DataFrame for Gym data only**

In [75]:
kl_gym = kl_grouped[["Neighborhoods","Gym"]]

In [76]:
kl_gym.head()

,Neighborhoods,Gym
0,Addison,0.00
1,Algonquin,0.00
2,Arlington Heights,0.01
3,Aurora,0.04
4,Barrington,0.00


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Chicago IL into 5 clusters.

In [77]:
# set number of clusters
kclusters = 5

kl_clustering = kl_gym.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 2, 0, 0, 0, 0, 0, 1])

In [78]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_gym.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [79]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Gym,Cluster Labels
0,Addison,0.00,0
1,Algonquin,0.00,0
2,Arlington Heights,0.01,4
3,Aurora,0.04,2
4,Barrington,0.00,0


In [80]:
# merge austin_grouped with austin_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(142, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Addison,0.00,0,41.946669,-87.702863
1,Algonquin,0.00,0,42.006096,-87.762853
2,Arlington Heights,0.01,4,41.926742,-87.642707
3,Aurora,0.04,2,41.883862,-88.208654
4,Barrington,0.00,0,41.989599,-87.735330


In [81]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(142, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Addison,0.000000,0,41.946669,-87.702863
35,Deerfield,0.000000,0,42.169050,-87.846210
125,South Chicago Heights,0.000000,0,41.912930,-88.177300
37,Dolton,0.000000,0,41.644723,-87.623586
123,Schaumburg,0.000000,0,42.288247,-87.873726
85,Medinah,0.000000,0,41.988976,-87.779016
40,East Dundee,0.000000,0,42.101560,-88.277340
84,Maywood,0.000000,0,41.925067,-87.816438
121,Round Lake Beach,0.000000,0,41.499829,-87.635578
82,Markham,0.000000,0,41.987819,-87.775058


**Finally, let's visualize the resulting clusters**

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [83]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [84]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Addison,0.0,0,41.946669,-87.702863
35,Deerfield,0.0,0,42.169050,-87.846210
125,South Chicago Heights,0.0,0,41.912930,-88.177300
37,Dolton,0.0,0,41.644723,-87.623586
123,Schaumburg,0.0,0,42.288247,-87.873726
85,Medinah,0.0,0,41.988976,-87.779016
40,East Dundee,0.0,0,42.101560,-88.277340
84,Maywood,0.0,0,41.925067,-87.816438
121,Round Lake Beach,0.0,0,41.499829,-87.635578
82,Markham,0.0,0,41.987819,-87.775058


#### Cluster 1

In [85]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
101,Oak Brook,0.020000,1,41.900791,-87.626261
75,Lake Bluff,0.020000,1,41.959893,-87.659425
76,Lake Forest,0.020000,1,41.959893,-87.659425
100,Northlake,0.020000,1,41.900791,-87.626261
83,Matteson,0.017241,1,41.517100,-87.734720
102,Oak Forest,0.020000,1,41.900791,-87.626261
97,North Shore,0.016949,1,42.130590,-87.831540
66,Hoffman Estates,0.015625,1,42.062980,-88.122950
9,Bensenville,0.023256,1,41.956040,-87.941060
138,Wilmette,0.017857,1,42.105210,-87.733630


#### Cluster 2

In [86]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
68,Inverness,0.045455,2,42.124270,-88.078590
79,Lockport,0.048780,2,41.588960,-88.055910
91,Morton Grove,0.043478,2,41.910679,-88.155400
3,Aurora,0.040000,2,41.883862,-88.208654


#### Cluster 3

In [87]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
122,Sauk Village,0.027027,3,42.026170,-88.065420
127,Stickney,0.026316,3,42.012100,-88.175580
117,Riverside,0.030000,3,41.884250,-87.632450
133,Waukegan,0.030000,3,41.879420,-87.633130
135,Westchester,0.030000,3,41.884250,-87.632450
136,Wheaton,0.026667,3,42.139550,-87.915730
112,Park Ridge,0.030000,3,41.884250,-87.632450
110,Palos Heights,0.028571,3,41.703780,-87.829230
116,Riverdale,0.030000,3,41.877094,-87.638687
70,Justice,0.030000,3,41.926800,-87.673480


#### Cluster 4

In [89]:
kl_merged.loc[kl_merged['Cluster Labels'] == 4]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
15,Bridgeview,0.010000,4,41.917990,-87.648530
111,Palos Hills,0.010000,4,41.727790,-87.715450
12,Bloomingdale,0.010000,4,41.899380,-87.624140
11,Berwyn,0.010000,4,41.974029,-87.871099
137,Wheeling,0.010000,4,42.075680,-87.707650
10,Berkeley,0.010000,4,41.816493,-87.600638
2,Arlington Heights,0.010000,4,41.926742,-87.642707
21,Burr Ridge,0.010000,4,41.703150,-87.779320
14,Bolingbrook,0.012048,4,41.698210,-88.079640
34,Darien,0.010000,4,41.946220,-87.716960


#### Observations

Cluster 0 takes a heavy portion of the neighboorhoods and is widley scattered across near inner city and outer suburbs. It is by far the most diverse and has the lowest amount of gyms. I do believe Foursquare is missing alot of data over gyms pertaining Cluster 0. However, I will still attempt to make an analysis based off this. We can come up with areas to avoid, such as Clusters 1 and 3 which have moderate competetition and would remotely difficult to open a gym there. Cluster 2 is the smallest and most saturated with gyms out of any cluster, so it would be best to avoid this area all together. Cluster 4 has lower amount of gyms overall and could be a good opprotunity to build. Overall, I dont think that Cluster 0 is accurate in the sense that it doesnt have any gyms in all those neighboorhoods, especially near other clusters that have gyms. But I did get to see areas to avoid and a cluster that had low competetition. 